In [1]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import torch.nn as nn
from torch.nn import functional as F
from torch.autograd import Variable

In [3]:
import dlc_practical_prologue as prologue

train_input, train_target, test_input, test_target = \
    prologue.load_data(one_hot_labels = True, normalize = True, flatten = False)

* Using MNIST
** Reduce the data-set (use --full for the full thing)
** Use 1000 train and 1000 test samples


In [40]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=5)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5)
        self.fc1 = nn.Linear(256, 200)
        self.fc2 = nn.Linear(200, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), kernel_size=3, stride=3))
        x = F.relu(F.max_pool2d(self.conv2(x), kernel_size=2, stride=2))
        x = F.relu(self.fc1(x.view(-1, 256)))
        x = self.fc2(x)
        return x

In [5]:
train_input, train_target = Variable(train_input), Variable(train_target)
model = Net()

## Training Function

In [24]:
def train_model(model, train_input, train_target, mini_batch_size, eta=1e-1, 
                num_epochs=25, criterion=nn.MSELoss()):
   
    for e in range(0, num_epochs):
        sum_loss = 0
        # We do this with mini-batches
        for b in range(0, train_input.size(0), mini_batch_size):
            output = model(train_input.narrow(0, b, mini_batch_size))
            loss = criterion(output, train_target.narrow(0, b, mini_batch_size))
            sum_loss = sum_loss + loss.item()
            model.zero_grad()
            loss.backward()
            for p in model.parameters():
                p.data.sub_(eta * p.grad.data)
        print("Loss",e, sum_loss)

In [25]:
train_model(model, train_input, train_target, 100)

Loss 0 0.17489974480122328
Loss 1 0.1647532070055604
Loss 2 0.15682137198746204
Loss 3 0.15737433265894651
Loss 4 0.15803939662873745
Loss 5 0.15815046429634094
Loss 6 0.15859481040388346
Loss 7 0.14998853206634521
Loss 8 0.15034357644617558
Loss 9 0.1533521907404065
Loss 10 0.1434680251404643
Loss 11 0.14030923042446375
Loss 12 0.15024878084659576
Loss 13 0.14576242584735155
Loss 14 0.14165062177926302
Loss 15 0.13995695859193802
Loss 16 0.13601051550358534
Loss 17 0.13578791078180075
Loss 18 0.13432730454951525
Loss 19 0.13639160990715027
Loss 20 0.1342464480549097
Loss 21 0.1295823808759451
Loss 22 0.1309264963492751
Loss 23 0.13282429054379463
Loss 24 0.12577392999082804


## Test Error

In [26]:
def compute_nb_errors(model, test_input, test_target, mini_batch_size):
    nb_error = 0.0
    for b in range(0, test_input.size(0), mini_batch_size):
        output_test = model(test_input.narrow(0, b, mini_batch_size))
        i, predicted_classes = output_test.max(1)
        #print(predicted_classes)
        for c in range(0,mini_batch_size):
            if test_target[b + c, predicted_classes[c]] <= 0:
                nb_error +=1
    return nb_error

In [27]:
test_input, test_target = Variable(test_input), Variable(test_target)

In [28]:
compute_nb_errors(model,test_input,test_target,100)

85.0

In [39]:
for k in range(10):
    model = Net()
    train_model(model, train_input, train_target, mini_batch_size=200)
    nb_test_errors = compute_nb_errors(model, test_input, test_target, mini_batch_size=200)
    print('test error Net {:0.2f}% '.format((100 * nb_test_errors) / test_input.size(0)))

Loss 0 0.4810798093676567
Loss 1 0.4284026026725769
Loss 2 0.4097045436501503
Loss 3 0.3931649923324585
Loss 4 0.37750063091516495
Loss 5 0.36229298263788223
Loss 6 0.34756774455308914
Loss 7 0.3334425762295723
Loss 8 0.32016026601195335
Loss 9 0.3076443709433079
Loss 10 0.295965775847435
Loss 11 0.2851091958582401
Loss 12 0.27500100433826447
Loss 13 0.2656019739806652
Loss 14 0.2568773031234741
Loss 15 0.24865758791565895
Loss 16 0.24130580574274063
Loss 17 0.23415334895253181
Loss 18 0.24070346355438232
Loss 19 0.28150344640016556
Loss 20 0.23322124034166336
Loss 21 0.21291149407625198
Loss 22 0.20926125720143318
Loss 23 0.2072732038795948
Loss 24 0.23557151854038239
test error Net 36.40% 
Loss 0 0.4673112630844116
Loss 1 0.4227360785007477
Loss 2 0.40531057864427567
Loss 3 0.38897818326950073
Loss 4 0.37300970405340195
Loss 5 0.35730377584695816
Loss 6 0.34221170097589493
Loss 7 0.3279404640197754
Loss 8 0.3146047778427601
Loss 9 0.30220959708094597
Loss 10 0.29080580174922943
Loss 

## Influence of the number of hidden units

In [50]:
class Net1(nn.Module):
    def __init__(self,hidden):
        super(Net1, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=5)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5)
        self.fc1 = nn.Linear(256, hidden)
        self.fc2 = nn.Linear(hidden, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), kernel_size=3, stride=3))
        x = F.relu(F.max_pool2d(self.conv2(x), kernel_size=2, stride=2))
        x = F.relu(self.fc1(x.view(-1, 256)))
        x = self.fc2(x)
        return x

In [55]:
for n_hidden in [10,50,200,500,1000]:
    modele = Net1(n_hidden)
    train_model(model, train_input, train_target, mini_batch_size=100)
    nb_test_errors = compute_nb_errors(model, test_input, test_target, mini_batch_size=100)
    print('Number hidden units:',n_hidden)
    print('test error Net1 for {:0.2f}% '.format((100 * nb_test_errors) / test_input.size(0)))

Loss 0 0.1249152161180973
Loss 1 0.14082168135792017
Loss 2 0.12776660732924938
Loss 3 0.1253474112600088
Loss 4 0.12718419451266527
Loss 5 0.12109488155692816
Loss 6 0.11611191555857658
Loss 7 0.11925851739943027
Loss 8 0.12263297755271196
Loss 9 0.11591476760804653
Loss 10 0.11490097176283598
Loss 11 0.11930596921592951
Loss 12 0.11484304629266262
Loss 13 0.11384119000285864
Loss 14 0.11704486701637506
Loss 15 0.11057753209024668
Loss 16 0.10745494347065687
Loss 17 0.10853250417858362
Loss 18 0.10845715831965208
Loss 19 0.10595536045730114
Loss 20 0.10480410512536764
Loss 21 0.10682323016226292
Loss 22 0.1059929532930255
Loss 23 0.10443603713065386
Loss 24 0.10716575663536787
Number hidden units: 10
test error Net1 for 7.00% 
Loss 0 0.1039607860147953
Loss 1 0.09965227637439966
Loss 2 0.09778857324272394
Loss 3 0.09790495689958334
Loss 4 0.0998331243172288
Loss 5 0.10394923854619265
Loss 6 0.10192398447543383
Loss 7 0.09590509254485369
Loss 8 0.0951544139534235
Loss 9 0.0952661670744

## Three convolutionnal layers

In [56]:
class Net2(nn.Module):
    def __init__(self,nb_hidden):
        super(Net2, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=5)
        self.conv2 = nn.Conv2d(32, 32, kernel_size=5)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=2)
        self.fc1 = nn.Linear(9 * 64, nb_hidden)
        self.fc2 = nn.Linear(nb_hidden, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), kernel_size=2))
        x = F.relu(F.max_pool2d(self.conv2(x), kernel_size=2))
        x = F.relu(self.conv3(x))
        x = F.relu(self.fc1(x.view(-1, 9 * 64)))
        x = self.fc2(x)
        return x

In [58]:
for k in range(10):
    model = Net2(200)
    train_model(model, train_input, train_target, mini_batch_size=200)
    nb_test_errors = compute_nb_errors(model, test_input, test_target, mini_batch_size=200)
    print('test error Net {:0.2f}% '.format((100 * nb_test_errors) / test_input.size(0)))

Loss 0 0.5040318518877029
Loss 1 0.45988039672374725
Loss 2 0.4415154233574867
Loss 3 0.4312039315700531
Loss 4 0.4227749705314636
Loss 5 0.4145638644695282
Loss 6 0.40598976612091064
Loss 7 0.3967903032898903
Loss 8 0.38687441498041153
Loss 9 0.37621260434389114
Loss 10 0.3646737188100815
Loss 11 0.35241807252168655
Loss 12 0.33976249396800995
Loss 13 0.32705460488796234
Loss 14 0.3146429471671581
Loss 15 0.3029722683131695
Loss 16 0.29223787412047386
Loss 17 0.2826753333210945
Loss 18 0.2742007002234459
Loss 19 0.2664363197982311
Loss 20 0.2591361738741398
Loss 21 0.2526171989738941
Loss 22 0.24659521505236626
Loss 23 0.24265699461102486
Loss 24 0.2509826421737671
test error Net 27.50% 
Loss 0 0.5176264718174934
Loss 1 0.47917240858078003
Loss 2 0.4574388861656189
Loss 3 0.4443845748901367
Loss 4 0.43628543615341187
Loss 5 0.43045322597026825
Loss 6 0.42533665895462036
Loss 7 0.42017336189746857
Loss 8 0.41460835188627243
Loss 9 0.4085366502404213
Loss 10 0.40184085816144943
Loss 11 